<h2>Перед запуском</h2>

<h4>Переход в нужную папку:</h4>

In [1]:
cd C:\Projects\Calc2

C:\Projects\Calc2


In [2]:
Dir = './In'

<h4>Установка библиотек:</h4>

In [14]:
!pip install pandas openpyxl xlsxwriter xlrd tqdm IPython ipywidgets


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<h4>Подключение библиотек:</h4>

In [3]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils import get_column_letter
import os
from IPython.display import display
from tqdm.notebook import tqdm
from multiprocessing import Pool
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

<h4>Запись файла pkl в Excel (Если нужно)</h4>

In [4]:
for file in os.listdir(Dir):
    if file[:-4] + '.xlsx' not in os.listdir('./Original') and file[-4:] == '.pkl':
        daf = pd.read_pickle(f'{Dir}/{file}')
        writer = pd.ExcelWriter(f'./Original/{file[:-4]}.xlsx', engine='xlsxwriter')
        daf.to_excel(writer, "Result")
        writer.close()

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: './Original'

<h2>Создание таблицы Result</h2>

<h4>Чтение файлов Excel</h4>

In [4]:
Files = os.listdir(Dir)
DATA = []
OutFiles = []
Dict = []
StatisticInData = pd.DataFrame()
StatisticInData.loc[0, 'Data'] = 'In Raw Data'
StatisticInData.loc[1, 'Data'] = 'In Finance Data'
NumberOfRows = 0

if 'Out' not in os.listdir():
    os.mkdir('Out')

from ReadFile import readFiles

tmp = []
for i in Files:
    if i[-4:]=='.pkl':
        tmp.append(i)
Files = tmp

FilesPB = tqdm(total=len(Files), desc="Files")

tmp = []
for file in Files:
    tmp.append((file, Dir))

for Number, filepath, df in Pool(min(len(Files), os.cpu_count())).map(readFiles, tmp):
    if df.empty:
        display(f'{filepath} error!')
    else:
        NumberOfRows += Number
        DATA.append(df[['PolicyTransactionTypeTitle', 'PolicyTransactionStatusTitle', 'IsReversal', 'HasBeenReversed']])
        OutFiles.append(filepath)
        Dict.append(df)
    FilesPB.update(1)
    FilesPB.display()
FilesPB.close()
StatisticInData.loc[0, 'RowsInData'] = NumberOfRows
DATA = pd.concat(DATA)
tmp = pd.concat(Dict)
StatisticInData.loc[0, 'TransactionsInData'] = len(DATA)

Dict = {k:v for (k,v) in zip(OutFiles, Dict)}

Files:   0%|          | 0/1 [00:00<?, ?it/s]

<h4>Прочтение файла fin</h4>

In [5]:
FinFiles = os.listdir('./Fin')
ExcelFile = pd.ExcelFile(f'./Fin/{FinFiles[0]}')
Headers = ['Facility', 'Policy Number','Transaction ID', 'Type', 'Status', 'Broker', 'Brokerage', 'Self Fee',
           'Effective Date', 'Inception Date', 'Expiry Date', 'Term Premium', 'Retained Commission']
df = ExcelFile.parse(ExcelFile.sheet_names[0])[Headers]
ExcelFile.close()
StatisticInData.loc[1, 'RowsInData'] = len(df)
tmp = df[Headers[:3]+['Term Premium']]
tmp = tmp.groupby(Headers[:3]).sum().reset_index()
df = df.groupby(Headers[:3]).agg('first').reset_index()
for i, row in tmp.iterrows():
    df.loc[[i], 'Term Premium'] = row['Term Premium']
StatisticInData.loc[1, 'TransactionsInData'] = len(df)
OutRows = df
OutRows

,Facility,Policy Number,Transaction ID,Type,Status,Broker,Brokerage,Self Fee,Effective Date,Inception Date,Expiry Date,Term Premium,Retained Commission
0,BRO-GLA-12,BRO-10091,17542,New Business,Bound,INTAS Insurance Services,694.80,0.0,2013-01-14,2013-01-14,2013-06-30,4632.00,0.00
1,BRO-GLA-12,BRO-10629,17544,New Business,Bound,INTAS Insurance Services,59.99,0.0,2013-01-18,2013-01-18,2013-06-30,399.96,0.00
2,BRO-GLA-12,BRO-12349,14473,Endorsement,Bound,Rural Affinity Bad Debtors,0.00,0.0,2013-04-03,2013-03-13,2013-06-30,3630.00,0.00
3,BRO-GLA-12,BRO-12670,11746,Endorsement Replaced,Bound Superceded,Austbrokers Dalby Pty Ltd,928.42,150.0,2013-06-30,2013-03-22,2013-06-30,6189.49,0.00
4,BRO-GLA-12,BRO-12670,11747,Reversal,Bound Superceded,Austbrokers Dalby Pty Ltd,-928.42,-150.0,2013-06-30,2013-03-22,2013-06-30,-6189.49,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25714,BRO-TAL-19,BRO-87490,97303,Endorsement,Bound,Ausure Yorke Peninsula (John Webb & Associate),410.23,50.0,2019-10-11,2019-09-04,2020-06-30,2344.16,234.41
25715,BRO-TAL-19,BRO-89841,99574,Endorsement Replaced,Bound Superceded,"Nutrien Ag Solutions, Lavington",118.30,50.0,2019-11-04,2019-06-30,2020-01-30,788.64,78.86
25716,BRO-TAL-19,BRO-89841,111422,Reversal,Bound Superceded,"Nutrien Ag Solutions, Lavington",-118.30,-50.0,2019-11-04,2019-06-30,2020-01-30,-788.64,-78.86
25717,BRO-TAL-19,BRO-90323,100354,Endorsement,Bound,"CIS Community Insurance Solutions Pty Ltd, Ben...",1630.67,50.0,2019-11-14,2019-10-03,2020-06-30,9318.14,931.82


<h4>Добавление новых столбцов из файла Fin</h4>

In [6]:
for filepath in Dict.keys():
    Dict[filepath]['Facility Fin'] = pd.Series(['']*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Policy Number Fin'] = pd.Series(['']*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Transaction ID Fin'] = pd.Series([np.nan]*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Type Fin'] = pd.Series(['']*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Status Fin'] = pd.Series(['']*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Broker Fin'] = pd.Series(['']*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Brokerage Fin'] = pd.Series([np.nan]*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Self Fee Fin'] = pd.Series([np.nan]*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Effective Date Fin'] = pd.Series(['']*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Inception Date Fin'] = pd.Series(['']*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Expiry Date Fin'] = pd.Series(['']*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Term Premium Fin'] = pd.Series([np.nan]*len(Dict[filepath]), index=Dict[filepath].index)
    Dict[filepath]['Retained Commission Fin'] = pd.Series([np.nan]*len(Dict[filepath]), index=Dict[filepath].index)

<h4>Заполнение новых столбцов</h4>

In [7]:
Files = os.listdir(Dir)
Matched = {}
for i, row in tqdm(OutRows.iterrows(),
                  total=len(OutRows),
                  desc='Rows: '):
    filepath = f'{row.iloc[0]}.pkl'.replace('-', '_')
    if row['Facility'] not in Matched.keys():
        Matched[row['Facility']] = []
    if filepath not in Files:
        #display(f'{filepath} is missing')
        continue
    index = (Dict[filepath].PolicyTransactionID==row.iloc[2])
    if row['Transaction ID'] not in Matched[row['Facility']]:
        Matched[row['Facility']].append(row['Transaction ID'])
    Dict[filepath].loc[index, 'Facility Fin'] = row.iloc[0]
    Dict[filepath].loc[index, 'Policy Number Fin'] = row.iloc[1]
    Dict[filepath].loc[index, 'Transaction ID Fin'] = row.iloc[2]
    Dict[filepath].loc[index, 'Type Fin'] = row.iloc[3]
    Dict[filepath].loc[index, 'Status Fin'] = row.iloc[4]
    Dict[filepath].loc[index, 'Broker Fin'] = row.iloc[5]
    Dict[filepath].loc[index, 'Brokerage Fin'] = row.iloc[6]
    Dict[filepath].loc[index, 'Self Fee Fin'] = row.iloc[7]
    Dict[filepath].loc[index, 'Effective Date Fin'] = row.iloc[8]
    Dict[filepath].loc[index, 'Inception Date Fin'] = row.iloc[9]
    Dict[filepath].loc[index, 'Expiry Date Fin'] = row.iloc[10]
    Dict[filepath].loc[index, 'Term Premium Fin'] = row.iloc[11]
    Dict[filepath].loc[index, 'Retained Commission Fin'] = row.iloc[12]

Rows:   0%|          | 0/25719 [00:00<?, ?it/s]

<h4>Уничтожение дубликатов и сортировка</h4>

In [8]:
DATA = DATA.drop_duplicates().sort_values(['PolicyTransactionTypeTitle', 'PolicyTransactionStatusTitle', 'IsReversal', 'HasBeenReversed']).reset_index().drop('index', axis=1)
DATA

,PolicyTransactionTypeTitle,PolicyTransactionStatusTitle,IsReversal,HasBeenReversed
0,Cancellation,Bound Superceded,False,True
1,Cancellation,Bound Superceded,True,False
2,Cancellation,Cancelled,False,False
3,Cancellation,Not Taken Up,False,False
4,Cancellation,Withdrawn,False,False
5,Endorsement,Bound,False,False
6,Endorsement,Bound Superceded,False,False
7,Endorsement,Bound Superceded,False,True
8,Endorsement,Bound Superceded,True,False
9,Endorsement,Declined,False,False


<h4>Добавление столбцов начала полиса и конца</h4>

In [9]:
Headers = ['FacilityCode', 'PolicyNumber', 'PolicyID', 'CropYear', 'YearPolicyTransactionID',
               'PolicyTransactionID', 'PolicyTransactionTypeTitle',
               'PolicyTransactionStatusTitle', 'BrokerName', 'InsuredName', 'CoverTypeName',
               'SeasonName', 'EventName', 'InceptionDate']

DATA['Starting state'] = pd.Series([0]*len(DATA), DATA.index)
DATA['Ending state'] = pd.Series([0]*len(DATA), DATA.index)
DATA['Any state'] = pd.Series([0]*len(DATA), DATA.index)
NMatch = {}
DF = []

for filepath in Dict.keys():
    Dict[filepath]['AfterOrOnFRD'] = pd.Series([0]*len(Dict[filepath]), index=Dict[filepath].index)
    for i, row in Dict[filepath].iterrows():
        if row['FacilityCode'] in Matched.keys():
            if row['PolicyTransactionID'] not in Matched[row['FacilityCode']]:
                if row['FacilityCode'] not in NMatch.keys():
                    NMatch[row['FacilityCode']] = []
                NMatch[row['FacilityCode']].append(row['PolicyTransactionID'])
        if row.loc['FinalRevisionDate']:
            if (row.loc['EffectiveDate'] - row.loc['FinalRevisionDate']).days >= 0:
                Dict[filepath].loc[i, 'AfterOrOnFRD'] = 1

for filepath in Dict.keys():
    DF.append(Dict[filepath])
DF = pd.concat(DF)
DF.sort_values(Headers[:6]).reset_index().drop('index', axis=1)
DF['Starting state'] = pd.Series([0]*len(Dict[filepath]), index=Dict[filepath].index)
DF['Ending state'] = pd.Series([0]*len(Dict[filepath]), index=Dict[filepath].index)
DF['Paid'] = pd.Series([0]*len(Dict[filepath]), index=Dict[filepath].index)
DF['Paid Police'] = pd.Series([0]*len(Dict[filepath]), index=Dict[filepath].index)
DF['Invoiced'] = pd.Series([0]*len(Dict[filepath]), index=Dict[filepath].index)
DF['Invoiced Police'] = pd.Series([0]*len(Dict[filepath]), index=Dict[filepath].index)
DF['Number of transaction'] = pd.Series([0]*len(Dict[filepath]), index=Dict[filepath].index)
DF['Sum of transaction'] = pd.Series([0]*len(Dict[filepath]), index=Dict[filepath].index)

for i, row in tqdm(DF.iterrows(),
                  total=len(DF),
                  desc='Rows:'):
    if i > 0:
        if (DF.iloc[i-1]['YearPolicyTransactionID'] != row.loc['YearPolicyTransactionID']) or (DF.iloc[i-1]['CropYear'] != row.loc['CropYear']) or (DF.iloc[i-1]['PolicyNumber'] != row.loc['PolicyNumber']):
            DF.loc[[i], 'Starting state'] = 1
            DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Starting state'] += 1
    else:
        DF.loc[[i], 'Starting state'] = 1
        DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Starting state'] += 1
    
    if i < len(DF)-1:
        if (DF.iloc[i+1]['YearPolicyTransactionID'] != row.loc['YearPolicyTransactionID']) or (DF.iloc[i+1]['CropYear'] != row.loc['CropYear']) or (DF.iloc[i+1]['PolicyNumber'] != row.loc['PolicyNumber']):
            DF.loc[[i], 'Ending state'] = 1
            DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Ending state'] += 1
    else:
        DF.loc[[i], 'Ending state'] = 1
        DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Ending state'] += 1
    DF.loc[[i], 'Premium'] = float(row['Premium'])
    DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Any state'] += 1



Rows::   0%|          | 0/11031 [00:00<?, ?it/s]

In [10]:
from Flags import Flags

for filepath in Dict.keys():
    Dict[filepath] = DF[DF['FacilityCode']==filepath[:-4].replace('_', '-')]
    
tmp = []
for filepath, df in tqdm(Pool(min(len(Files), os.cpu_count())).map(Flags, [(file, Dict[file]) for file in Dict.keys()]),
                  total=len(Dict),
                  desc='Files:'):
    tmp.append(df)
    Dict[filepath] = df

DF = pd.concat(tmp)
DF = DF.reset_index().drop('index', axis=1)

Files::   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
DATA.loc[len(DATA), 'PolicyTransactionTypeTitle'] = 'Total:'
DATA.loc[[len(DATA)-1], 'PolicyTransactionStatusTitle'] = ''
DATA.loc[[len(DATA)-1], 'Starting state'] = DATA['Starting state'].sum()
DATA.loc[[len(DATA)-1], 'Ending state'] = DATA['Ending state'].sum()
DATA.loc[[len(DATA)-1], 'Any state'] = DATA['Any state'].sum()
DATA

,PolicyTransactionTypeTitle,PolicyTransactionStatusTitle,IsReversal,HasBeenReversed,Starting state,Ending state,Any state
0,Cancellation,Bound Superceded,False,True,0.0,0.0,2.0
1,Cancellation,Bound Superceded,True,False,0.0,1.0,88.0
2,Cancellation,Cancelled,False,False,0.0,24.0,24.0
3,Cancellation,Not Taken Up,False,False,0.0,1.0,1.0
4,Cancellation,Withdrawn,False,False,0.0,0.0,2.0
5,Endorsement,Bound,False,False,0.0,1382.0,1403.0
6,Endorsement,Bound Superceded,False,False,1.0,899.0,2657.0
7,Endorsement,Bound Superceded,False,True,3.0,0.0,216.0
8,Endorsement,Bound Superceded,True,False,0.0,0.0,218.0
9,Endorsement,Declined,False,False,0.0,0.0,2.0


<h4>Создание таблицы статистики по таблицам Fin и Data</h4>

In [12]:
Statistic = pd.DataFrame()
def CalcStat(Statistic, DF, Matched, NMatch, OutRows):
    list = DF['FacilityCode'].drop_duplicates().to_list() + OutRows['Facility'].drop_duplicates().to_list()
    Statistic['Facility'] = pd.Series(list, index=range(0, len(list))).drop_duplicates()
    Statistic = Statistic.sort_values('Facility').reset_index().drop('index', axis=1)
    
    Statistic['In Finance Data'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Raw Data'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Both Datasets'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Finance Data Only'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Raw Data Only'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Raw Data Only - Premium=0 - Count'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Raw Data Only - Premium<>0 - Count'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Raw Data Only - Premium<>0 - Sum'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Raw Data Only - Count Not Invoiced'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Raw Data Only - Count Invoiced'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Raw Data Only - SumPremiumInvoiced'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Both Datasets - Not Invoiced - Count'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Both Datasets - Invoiced - Count'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Both Datasets - Invoiced - SumPremium'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Both Datasets - Paid - Count'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    Statistic['In Both Datasets - Paid - SumTermPremiumFin'] = pd.Series([0]*len(Statistic), index=Statistic.index)
    
    for i, row in tqdm(Statistic.iterrows(),
                      total=len(Statistic),
                      desc='Facilities:'):
        Statistic.loc[[i], 'In Finance Data'] = len(OutRows[OutRows['Facility']==row.iloc[0]])
        Statistic.loc[[i], 'In Raw Data'] = len(DF[DF['FacilityCode']==row.iloc[0]])
        Statistic.loc[[i], 'In Both Datasets'] = len(Matched[row['Facility']])
        Statistic.loc[[i], 'In Finance Data Only'] = Statistic.loc[i, 'In Finance Data']-Statistic.loc[i, 'In Both Datasets']
        Statistic.loc[[i], 'In Raw Data Only'] = len(NMatch[row['Facility']]) if row['Facility'] in NMatch.keys() else 0
        if row['Facility'] in NMatch.keys():
            Statistic.loc[[i], 'In Raw Data Only - Premium=0 - Count'] = len(DF[(pd.Series([i in NMatch[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Premium']==0)])
            Statistic.loc[[i], 'In Raw Data Only - Premium<>0 - Count'] = len(DF[(pd.Series([i in NMatch[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Premium']!=0)])
            Statistic.loc[[i], 'In Raw Data Only - Premium<>0 - Sum'] = round(DF[(pd.Series([i in NMatch[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Premium']!=0)]['Premium'].sum(), 2)
            Statistic.loc[[i], 'In Raw Data Only - Count Not Invoiced'] = len(DF[(pd.Series([i in NMatch[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Invoiced']==0)])
            Statistic.loc[[i], 'In Raw Data Only - Count Invoiced'] = len(DF[(pd.Series([i in NMatch[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Invoiced']==1)])
            Statistic.loc[[i], 'In Raw Data Only - SumPremiumInvoiced'] = round(DF[(pd.Series([i in NMatch[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Invoiced']==1)]['Premium'].sum(), 2)
        else:
            Statistic.loc[[i], 'In Raw Data Only - Premium=0 - Count'] = 0
            Statistic.loc[[i], 'In Raw Data Only - Premium<>0 - Count'] = 0
            Statistic.loc[[i], 'In Raw Data Only - Premium<>0 - Sum'] = 0
            Statistic.loc[[i], 'In Raw Data Only - Count Not Invoiced'] = 0
            Statistic.loc[[i], 'In Raw Data Only - Count Invoiced'] = 0
            Statistic.loc[[i], 'In Raw Data Only - SumPremiumInvoiced'] = 0
            
        Statistic.loc[[i], 'In Both Datasets - Not Invoiced - Count'] = len(DF[(pd.Series([i in Matched[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Invoiced']==0)])
        Statistic.loc[[i], 'In Both Datasets - Invoiced - Count'] = len(DF[(pd.Series([i in Matched[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Invoiced']==1)])
        Statistic.loc[[i], 'In Both Datasets - Invoiced - SumPremium'] = round(DF[(pd.Series([i in Matched[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Invoiced']==1)]['Premium'].sum(), 2)
        Statistic.loc[[i], 'In Both Datasets - Paid - Count'] = len(DF[(pd.Series([i in Matched[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Paid']==1)])
        Statistic.loc[[i], 'In Both Datasets - Paid - SumTermPremiumFin'] = round(DF[(pd.Series([i in Matched[row['Facility']] for i in DF['PolicyTransactionID']])) & (DF['FacilityCode']==row.iloc[0]) & (DF['Paid']==1)]['Premium'].sum(), 2)
    Statistic.loc[len(Statistic), 'Facility'] = 'Total:'
    Statistic.loc[[len(Statistic)-1], 'In Finance Data'] = Statistic['In Finance Data'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Raw Data'] = Statistic['In Raw Data'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Both Datasets'] = Statistic['In Both Datasets'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Finance Data Only'] = Statistic['In Finance Data Only'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Raw Data Only'] = Statistic['In Raw Data Only'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Raw Data Only - Premium=0 - Count'] = Statistic['In Raw Data Only - Premium=0 - Count'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Raw Data Only - Premium<>0 - Count'] = Statistic['In Raw Data Only - Premium<>0 - Count'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Raw Data Only - Premium<>0 - Sum'] = Statistic['In Raw Data Only - Premium<>0 - Sum'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Raw Data Only - Count Not Invoiced'] = Statistic['In Raw Data Only - Count Not Invoiced'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Raw Data Only - Count Invoiced'] = Statistic['In Raw Data Only - Count Invoiced'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Raw Data Only - SumPremiumInvoiced'] = Statistic['In Raw Data Only - SumPremiumInvoiced'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Both Datasets - Not Invoiced - Count'] = Statistic['In Both Datasets - Not Invoiced - Count'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Both Datasets - Invoiced - Count'] = Statistic['In Both Datasets - Invoiced - Count'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Both Datasets - Invoiced - SumPremium'] = Statistic['In Both Datasets - Invoiced - SumPremium'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Both Datasets - Paid - Count'] = Statistic['In Both Datasets - Paid - Count'].sum()
    Statistic.loc[[len(Statistic)-1], 'In Both Datasets - Paid - SumTermPremiumFin'] = Statistic['In Both Datasets - Paid - SumTermPremiumFin'].sum()
    return Statistic

Statistic = CalcStat(Statistic, DF, Matched, NMatch, OutRows)
Statistic

Facilities::   0%|          | 0/13 [00:00<?, ?it/s]

,Facility,In Finance Data,In Raw Data,In Both Datasets,In Finance Data Only,In Raw Data Only,In Raw Data Only - Premium=0 - Count,In Raw Data Only - Premium<>0 - Count,In Raw Data Only - Premium<>0 - Sum,In Raw Data Only - Count Not Invoiced,In Raw Data Only - Count Invoiced,In Raw Data Only - SumPremiumInvoiced,In Both Datasets - Not Invoiced - Count,In Both Datasets - Invoiced - Count,In Both Datasets - Invoiced - SumPremium,In Both Datasets - Paid - Count,In Both Datasets - Paid - SumTermPremiumFin
0,BRO-GLA-12,2846.0,0.0,0.0,2846.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
1,BRO-GLA-13,2540.0,0.0,0.0,2540.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2,BRO-GLA-14,2251.0,0.0,0.0,2251.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
3,BRO-GLA-15,2264.0,0.0,0.0,2264.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
4,BRO-GLA-16,2199.0,0.0,0.0,2199.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
5,BRO-GLA-17,1655.0,0.0,0.0,1655.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
6,BRO-RAU-20,2490.0,0.0,0.0,2490.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
7,BRO-RAU-21,2373.0,0.0,0.0,2373.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
8,BRO-RAU-22,2800.0,11031.0,2800.0,0.0,8231.0,78.0,8153.0,2.929298e+08,7848.0,383.0,10006540.94,2.0,2798.0,63422597.0,2418.0,50955059.62
9,BRO-RAU-23,43.0,0.0,0.0,43.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00


<h4>Удаление строк</h4>

In [13]:
Deleted = []
def map_previous_and_next_transactionID(policy_df: pd.DataFrame) -> pd.DataFrame:
    # Add PreviousPolicyTransactionID and NextPolicyTransactionID fields
    # per policy
    temp_df = (
        policy_df.sort_values(by='PolicyTransactionID')
        .drop_duplicates(subset='PolicyTransactionID')
    )
    previous_ID_dict = dict(zip(
        temp_df['PolicyTransactionID'], temp_df['PolicyTransactionID'].shift(1)
    ))
    next_ID_dict = dict(zip(
        temp_df['PolicyTransactionID'], temp_df['PolicyTransactionID'].shift(-1)
    ))
    policy_df['PreviousPolicyTransactionID'] = policy_df['PolicyTransactionID'].map(previous_ID_dict)
    policy_df['NextPolicyTransactionID'] = policy_df['PolicyTransactionID'].map(next_ID_dict)
    return policy_df

quote_statuses = ['Not Taken Up', 'Withdrawn', 'Declined', 'Quote', 'Withdrawn']

# Filter out quotes and add on Next and Previous TransactionIDs.
# Quotes should be filtered out before mapping on Previous/Next transactions.
bound_data = (
    DF[~DF.PolicyTransactionStatusTitle.isin(quote_statuses)]
    .groupby(['PolicyNumber'])
    .apply(map_previous_and_next_transactionID)
    .reset_index(drop=True)
)

reversals = bound_data[bound_data.PolicyTransactionTypeTitle == 'Reversal'].PolicyTransactionID.unique()

filtered = bound_data[
    (~bound_data.IsReversal) 
    & (~bound_data.HasBeenReversed) 
    & (~bound_data.PolicyTransactionTypeTitle.isin(['Replaced', 'Reversal']))
    & ~(
        # To remove Cancellations that have been reversed.
        (bound_data.NextPolicyTransactionID.isin(reversals)) 
        & (bound_data.PolicyTransactionTypeTitle == 'Cancellation')
    )
]

cancellation_df = (
    filtered[filtered.PolicyTransactionTypeTitle == 'Cancellation']
    .drop_duplicates(subset='PolicyTransactionID')
    .sort_values(by='PolicyTransactionID')
)
# Cancellations
cancellation_trans = list(cancellation_df.PolicyTransactionID.unique()) 
# Transactions being cancelled
cancelled_trans = list(cancellation_df.PreviousPolicyTransactionID.unique()) 
tmp = filtered[~filtered.PolicyTransactionID.isin(cancellation_trans + cancelled_trans)]
Deleted.append(DF.loc[~DF.PolicyTransactionID.isin(tmp.PolicyTransactionID)])
DF = tmp.reset_index().drop('index', axis=1)


ForDeleting = {'Cancellation':[('Not Taken Up', False, False),
                  ('Bound Superceded', True, False),
                  ('Bound Superceded', False, True),
                  ('Withdrawn', False, False)],
              'Endorsement':[('Bound Superceded', False, True),
                  ('Bound Superceded', True, False),
                  ('Declined', False, False),
                  ('Not Taken Up', False, False),
                  ('Quote', False, False),
                  ('Withdrawn', False, False)],
              'New Business':[('Declined', False, False),
                  ('Bound Superceded', False, True),
                  ('Not Taken Up', False, False),
                  ('Quote', False, False),
                  ('Withdrawn', False, False)],
              'New Season':[('Declined', False, False),
                  ('Bound Superceded', False, True),
                  ('Not Taken Up', False, False),
                  ('Quote', False, False),
                  ('Withdrawn', False, False)],
              'Renewal':[('Not Taken Up', False, False),
                  ('Bound Superceded', False, True),
                  ('Withdrawn', False, False)],
              'Replaced':[('Bound', False, False),
                  ('Bound Superceded', False, False)],
              'Reversal':[('Bound', False, False),
                  ('Bound Superceded', False, False)]}
Deleted2 = pd.DataFrame()
for type in tqdm(ForDeleting.keys(),
                total=len(ForDeleting),
                desc='Types:'):
    for (status, isRev, hasBeenRev) in ForDeleting[type]:
        if not DF[(DF.PolicyTransactionTypeTitle==type) & (DF.PolicyTransactionStatusTitle==status) & (DF.IsReversal==isRev) & (DF.HasBeenReversed==hasBeenRev)].empty:
            tmpAll = DF[(DF.PolicyTransactionTypeTitle==type) & (DF.PolicyTransactionStatusTitle==status) & (DF.IsReversal==isRev) & (DF.HasBeenReversed==hasBeenRev)]
            for i, tmp in tmpAll.iterrows():
                if tmp['PolicyTransactionID'] in Matched[tmp['FacilityCode']]:
                    Matched[tmp['FacilityCode']].remove(tmp['PolicyTransactionID'])
                if tmp['PolicyTransactionID'] in NMatch[tmp['FacilityCode']]:
                    NMatch[tmp['FacilityCode']].remove(tmp['PolicyTransactionID'])
            Deleted.append(DF[(DF.PolicyTransactionTypeTitle==type) & (DF.PolicyTransactionStatusTitle==status) & (DF.IsReversal==isRev) & (DF.HasBeenReversed==hasBeenRev)])
            DF = DF[(DF.PolicyTransactionTypeTitle!=type) | (DF.PolicyTransactionStatusTitle!=status) | (DF.IsReversal!=isRev) | (DF.HasBeenReversed!=hasBeenRev)]

DF = DF.reset_index().drop('index', axis=1)
for filepath in Dict.keys():
    Dict[filepath] = DF[DF['FacilityCode']==filepath[:-4].replace('_', '-')]



Types::   0%|          | 0/7 [00:00<?, ?it/s]

<h4>Удаление полисов, начинающихся с Cancellation</h4>

In [51]:
Deleted2 = []
for filepath in Dict.keys():
    for i, row in Dict[filepath][(Dict[filepath]['Starting state']==1) & (Dict[filepath].PolicyTransactionTypeTitle=='Cancellation')].iterrows():
        Deleted2.append(Dict[filepath][(Dict[filepath]['YearPolicyTransactionID'] == row.loc['YearPolicyTransactionID']) & (Dict[filepath]['CropYear'] == row.loc['CropYear']) & (Dict[filepath]['PolicyNumber'] == row.loc['PolicyNumber']) & (Dict[filepath]['FacilityCode'] == row.loc['FacilityCode'])])
        Dict[filepath] = Dict[filepath][(Dict[filepath]['YearPolicyTransactionID'] != row.loc['YearPolicyTransactionID']) | (Dict[filepath]['CropYear'] != row.loc['CropYear']) | (Dict[filepath]['PolicyNumber'] != row.loc['PolicyNumber']) | (Dict[filepath]['FacilityCode'] != row.loc['FacilityCode'])]
if len(Deleted2):
    Deleted2 = pd.concat(Deleted2)
else:
    Deleted2 = pd.DataFrame()
    
display(Deleted2)

[]

<h4>Запись удаленных в файл</h4>

In [14]:
if len(Deleted):
    Deleted = pd.concat(Deleted)
    DelWrite = pd.ExcelWriter(f'./Out/RA_Pol_Tran_20230904_Deleted.xlsx', engine='xlsxwriter')
    Deleted.to_excel(DelWrite, "Deleted Rows")
    Deleted2.to_excel(DelWrite, "Deleted Polices")
    DelWrite.close()
    ExcelFile = openpyxl.load_workbook(f'.\\Out\\RA_Pol_Tran_20230904_Deleted.xlsx')
    for Sheet in ExcelFile:
        column_widths = []
        for i, cell in enumerate(Sheet[1]):
            cell = str(cell.value)
            column_widths.append(len(cell) * 1.2 + 2)
        for i, column_width in enumerate(column_widths, 1):
            Sheet.column_dimensions[get_column_letter(i)].width = column_width
    ExcelFile.save(f'.\\Out\\RA_Pol_Tran_20230904_Deleted.xlsx')
    ExcelFile.close()

<h4>Пересчет таблиц</h4>

In [ ]:
DATA1 = DATA.copy()

In [16]:
DATA['Starting state'] = pd.Series([0]*len(DATA), DATA.index)
DATA['Ending state'] = pd.Series([0]*len(DATA), DATA.index)
DATA['Any state'] = pd.Series([0]*len(DATA), DATA.index)
DF = []
for filepath in Dict.keys():
    DF.append(Dict[filepath])
DF = pd.concat(DF)
DF.sort_values(Headers[:6]).reset_index().drop('index', axis=1)

for i, row in tqdm(DF.iterrows(),
                  total=len(DF),
                  desc='Rows:'):
    if i > 0:
        if (DF.iloc[i-1]['YearPolicyTransactionID'] != row.loc['YearPolicyTransactionID']) or (DF.iloc[i-1]['CropYear'] != row.loc['CropYear']) or (DF.iloc[i-1]['PolicyNumber'] != row.loc['PolicyNumber']):
            DF.loc[[i], 'Starting state'] = 1
            DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Starting state'] += 1
    else:
        DF.loc[[i], 'Starting state'] = 1
        DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Starting state'] += 1
    
    if i < len(DF)-1:
        if (DF.iloc[i+1]['YearPolicyTransactionID'] != row.loc['YearPolicyTransactionID']) or (DF.iloc[i+1]['CropYear'] != row.loc['CropYear']) or (DF.iloc[i+1]['PolicyNumber'] != row.loc['PolicyNumber']):
            DF.loc[[i], 'Ending state'] = 1
            DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Ending state'] += 1
    else:
        DF.loc[[i], 'Ending state'] = 1
        DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Ending state'] += 1
    DF.loc[[i], 'Premium'] = float(row['Premium'])
    DATA.loc[(DATA.PolicyTransactionTypeTitle==row.PolicyTransactionTypeTitle) & (DATA.PolicyTransactionStatusTitle==row.PolicyTransactionStatusTitle) & (DATA.IsReversal==row.IsReversal) & (DATA.HasBeenReversed==row.HasBeenReversed), 'Any state'] += 1

for filepath in Dict.keys():
    Dict[filepath] = DF[DF['FacilityCode']==filepath[:-4].replace('_', '-')].reset_index().drop('index', axis=1)
    
tmp = []
for filepath, df in tqdm(Pool(min(len(Files), os.cpu_count())).map(Flags, [(file, Dict[file]) for file in Dict.keys()]),
                  total=len(Dict),
                  desc='Files:'):
    tmp.append(df)
    Dict[filepath] = df

DATA.loc[[len(DATA)-1], 'PolicyTransactionTypeTitle'] = 'Total:'
DATA.loc[[len(DATA)-1], 'PolicyTransactionStatusTitle'] = ''
DATA.loc[[len(DATA)-1], 'Starting state'] = DATA['Starting state'].sum()
DATA.loc[[len(DATA)-1], 'Ending state'] = DATA['Ending state'].sum()
DATA.loc[[len(DATA)-1], 'Any state'] = DATA['Any state'].sum()
DATA

DF = pd.concat(tmp)
DF = DF.reset_index().drop('index', axis=1)

Statistic2 = pd.DataFrame()
Statistic2 = CalcStat(Statistic2, DF, Matched, NMatch, OutRows)

display(Statistic2)
DATA

Rows::   0%|          | 0/6458 [00:00<?, ?it/s]

Files::   0%|          | 0/1 [00:00<?, ?it/s]

Facilities::   0%|          | 0/13 [00:00<?, ?it/s]

,Facility,In Finance Data,In Raw Data,In Both Datasets,In Finance Data Only,In Raw Data Only,In Raw Data Only - Premium=0 - Count,In Raw Data Only - Premium<>0 - Count,In Raw Data Only - Premium<>0 - Sum,In Raw Data Only - Count Not Invoiced,In Raw Data Only - Count Invoiced,In Raw Data Only - SumPremiumInvoiced,In Both Datasets - Not Invoiced - Count,In Both Datasets - Invoiced - Count,In Both Datasets - Invoiced - SumPremium,In Both Datasets - Paid - Count,In Both Datasets - Paid - SumTermPremiumFin
0,BRO-GLA-12,2846.0,0.0,0.0,2846.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
1,BRO-GLA-13,2540.0,0.0,0.0,2540.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
2,BRO-GLA-14,2251.0,0.0,0.0,2251.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
3,BRO-GLA-15,2264.0,0.0,0.0,2264.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
4,BRO-GLA-16,2199.0,0.0,0.0,2199.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
5,BRO-GLA-17,1655.0,0.0,0.0,1655.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
6,BRO-RAU-20,2490.0,0.0,0.0,2490.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
7,BRO-RAU-21,2373.0,0.0,0.0,2373.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
8,BRO-RAU-22,2800.0,6458.0,2800.0,0.0,8231.0,36.0,3949.0,1.218232e+08,3733.0,252.0,6193991.76,0.0,2473.0,51831151.94,2277.0,45819176.15
9,BRO-RAU-23,43.0,0.0,0.0,43.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00


,PolicyTransactionTypeTitle,PolicyTransactionStatusTitle,IsReversal,HasBeenReversed,Starting state,Ending state,Any state
0,Cancellation,Bound Superceded,False,True,0,0,0
1,Cancellation,Bound Superceded,True,False,0,0,0
2,Cancellation,Cancelled,False,False,0,0,0
3,Cancellation,Not Taken Up,False,False,0,0,0
4,Cancellation,Withdrawn,False,False,0,0,0
5,Endorsement,Bound,False,False,0,1403,1403
6,Endorsement,Bound Superceded,False,False,1,939,2652
7,Endorsement,Bound Superceded,False,True,0,0,0
8,Endorsement,Bound Superceded,True,False,0,0,0
9,Endorsement,Declined,False,False,0,0,0


<h4>Запись таблиц сочетаний</h4>

In [17]:
writer = pd.ExcelWriter('.\\Out\\Result.xlsx', engine='xlsxwriter')
DATA1.to_excel(writer, "Statistic Type_Status")
DATA.to_excel(writer, "StatType_StatAfDel")
StatisticInData.to_excel(writer, "StatisticInData")
Statistic.to_excel(writer, "Statistic Fin_Raw")
Statistic2.to_excel(writer, "StatFin_RawDel")
writer.close()
ExcelFile = openpyxl.load_workbook('.\\Out\\Result.xlsx')
for Sheet in ExcelFile:
    column_widths = []
    for i, cell in enumerate(Sheet[1]):
        cell = str(cell.value)
        column_widths.append(len(cell)*1.2+2)
    for i, column_width in enumerate(column_widths, 1):
        Sheet.column_dimensions[get_column_letter(i)].width = column_width
ExcelFile.save('.\\Out\\Result.xlsx')
ExcelFile.close()
print('OK')

OK


<h4>Запись отдельных файлов</h4>

In [18]:
DF = []
for filepath in tqdm(Dict.keys(), 
                     total=len(Dict.keys()), 
                     desc='Files: '):
    DF.append(Dict[filepath])
    writer = pd.ExcelWriter(f'./Out/RA_Pol_Tran_20230904_{filepath[:-4]}.xlsx', engine='xlsxwriter')
    Dict[filepath].to_excel(writer, "Result")
    writer.close()
    ExcelFile = openpyxl.load_workbook(f'.\\Out\\RA_Pol_Tran_20230904_{filepath[:-4]}.xlsx')
    for Sheet in ExcelFile:
        column_widths = []
        for i, cell in enumerate(Sheet[1]):
            cell = str(cell.value)
            column_widths.append(len(cell) * 1.2 + 2)
        for i, column_width in enumerate(column_widths, 1):
            Sheet.column_dimensions[get_column_letter(i)].width = column_width
    ExcelFile.save(f'.\\Out\\RA_Pol_Tran_20230904_{filepath[:-4]}.xlsx')
    ExcelFile.close()
DF = pd.concat(DF)
DF

Files:   0%|          | 0/1 [00:00<?, ?it/s]

,FacilityCode,PolicyNumber,PolicyID,CropYear,YearPolicyTransactionID,PolicyTransactionID,PolicyTransactionTypeTitle,PolicyTransactionStatusTitle,BrokerName,InsuredName,...,Starting state,Ending state,Paid,Paid Police,Invoiced,Invoiced Police,Number of transaction,Sum of transaction,PreviousPolicyTransactionID,NextPolicyTransactionID
0,BRO-RAU-22,BRO-100629,8017,2022/23,524894.0,524894,New Season,Bound Superceded,"Ensured Services, Jarrahdale",Harding Sawyer,...,1,0,0,1,0,1,1,4,NaN,526566.0
1,BRO-RAU-22,BRO-100629,8017,2022/23,524894.0,526566,Endorsement,Bound Superceded,"Ensured Services, Jarrahdale",Harding Sawyer,...,0,0,0,1,0,1,2,4,524894.0,532482.0
2,BRO-RAU-22,BRO-100629,8017,2022/23,524894.0,532482,Endorsement,Bound Superceded,"Ensured Services, Jarrahdale",Harding Sawyer,...,0,0,0,1,0,1,3,4,526566.0,536671.0
3,BRO-RAU-22,BRO-100629,8017,2022/23,524894.0,536671,Endorsement,Bound Superceded,"Ensured Services, Jarrahdale",Harding Sawyer,...,0,1,1,1,1,1,4,4,532482.0,NaN
4,BRO-RAU-22,BRO-104223,7612,2022/23,524638.0,524638,New Season,Bound Superceded,"Nutrien Ag Solutions, North East",B & K Witham Mundoora Farming,...,1,0,0,1,0,1,1,2,NaN,531394.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6453,BRO-RAU-22,BRO-85113,8338,2022/23,524418.0,532100,Endorsement,Bound,"Country Wide Insurance Brokers, Northam",Des Smith & Co,...,0,1,1,1,1,1,2,2,524418.0,NaN
6454,BRO-RAU-22,BRO-85957,7824,2022/23,523870.0,523870,New Season,Bound Superceded,"Country Wide Insurance Brokers, Perth",Garison Farms,...,1,0,0,0,0,1,1,2,NaN,532381.0
6455,BRO-RAU-22,BRO-85957,7824,2022/23,523870.0,532381,Endorsement,Bound,"Country Wide Insurance Brokers, Perth",Garison Farms,...,0,1,0,0,1,1,2,2,523870.0,NaN
6456,BRO-RAU-22,BRO-86980,8948,2022/23,524751.0,524751,New Season,Bound Superceded,EBM Insurance Brokers,Francis Cole,...,1,0,0,1,0,1,1,2,NaN,532835.0


<h4>Запись общего файла</h4>

In [19]:
writer = pd.ExcelWriter(f'./Out/RA_Pol_Tran_20230904_All_Data.xlsx', engine='xlsxwriter')
DF.to_excel(writer, "Result")
writer.close()
ExcelFile = openpyxl.load_workbook(f'.\\Out\\RA_Pol_Tran_20230904_All_Data.xlsx')
for Sheet in ExcelFile:
    column_widths = []
    for i, cell in enumerate(Sheet[1]):
        cell = str(cell.value)
        column_widths.append(len(cell) * 1.2 + 2)
    for i, column_width in enumerate(column_widths, 1):
        Sheet.column_dimensions[get_column_letter(i)].width = column_width
ExcelFile.save(f'.\\Out\\RA_Pol_Tran_20230904_All_Data.xlsx')
ExcelFile.close()
print('OK')

OK
